# PL05. Regresión Logística _Scratch_ (con Gradiente Descendente)

__Borja González Seoane. Aprendizaje Automático. Curso 2024-25__

En este *notebook* se procede a implementar un modelo de regresión logística desde cero (_scratch_), es decir, sin utilizar librerías Scikit-Learn. Se implementará el algoritmo de gradiente descendente para minimizar la función de coste.

En este *notebook* se utilizará simplemente un conjunto de datos _dummy_ para probar el modelo. La parte del ejercicio relativa a trabajar con el conjunto de datos Titanic se realizará en el siguiente *notebook* de la PL05, para contrastar también entre diferentes modelos.


In [82]:
import numpy as np
from sklearn.datasets import (  # Interesante para generar datos de prueba
    make_classification,
)


## Creación de datos _dummy_

In [83]:
data = make_classification(
    n_samples=100,
    n_features=5,
    )

display(data)

X = data[0]
y = data[1]

(array([[-9.06068712e-02, -1.01132361e+00, -1.28649849e+00,
          2.97371111e-01, -5.66609813e-01],
        [ 9.67732668e-01,  1.30427959e+00, -1.74249093e+00,
         -7.47164629e-01,  8.55168226e-01],
        [-1.87831391e-01,  6.17534434e-01,  6.82782130e-01,
          9.09646341e-02, -5.03341917e-02],
        [-4.17762000e-01, -7.01645989e-01, -1.26192759e+00,
          6.38517458e-01, -1.04517394e+00],
        [-1.03456090e+00, -3.32324274e-01,  2.46969742e-01,
          1.05225083e+00, -1.45654913e+00],
        [-4.72485235e-01, -1.24742791e+00,  1.74438052e+00,
          2.24605759e-01, -1.17599807e-01],
        [-4.31366219e-01, -1.02301490e+00, -5.82992768e-02,
          4.64042404e-01, -6.61445098e-01],
        [-5.63720894e-01,  1.08535951e+00, -9.95336673e-01,
          7.50616259e-01, -1.17288702e+00],
        [ 1.25781174e+00,  8.49357557e-01, -1.97904342e-01,
         -1.29537689e+00,  1.80521685e+00],
        [-1.83588132e-01, -2.56210892e-01, -1.70003662e+00,
    

## Implementación del modelo por partes

Primero se procede a implementar las funciones necesarias para el modelo de regresión logística. Se implementarán las siguientes funciones:

1. `predict`: $W*X + b$, donde $W$ es el vector de pesos y $b$ es el sesgo o _bias_.
2. `sigmoid`: $\frac{1}{1 + e^{-\text{predict}(X)}}$.
3. `loss`: $-\frac{1}{m} \sum_{i=1}^{m} y_i \log(\text{sigmoid}(X_i)) + (1 - y_i) \log(1 - \text{sigmoid}(X_i))$.
4. `dl_dw`, derivada de la función de coste respecto a los pesos: $\frac{1}{m} \sum_{i=1}^{m} (\text{sigmoid}(X_i) - y_i)X_i$.
5. `dl_db`, derivada de la función de coste respecto al sesgo: $\frac{1}{m} \sum_{i=1}^{m} (\text{sigmoid}(X_i) - y_i)$.
6. `update`, actualización de los pesos y el sesgo: $W = W - \alpha \text{dl\_dw}$ y $b = b - \alpha \text{dl\_db}$, siendo $\alpha$ la tasa de aprendizaje.
7. `fit`, función que, a partir de las piezas anteriores, entrena el modelo en un número de iteraciones dado.



In [84]:
#funciones prediccion

def predict(x, w, b):
    return np.dot(x, w) + b

def sigmoid(yhat):
    return 1/(1+np.exp(-yhat))

def loss(y, sigmoid):
    return -(y*np.log(sigmoid) + (1-y)*np.log(1-sigmoid)).mean()

def dldw(x, y, sigmoid):
    return (np.reshape(sigmoid-y, (x.shape[0], 1))*x).mean(axis=0)

def dldb(y, sigmoid):
    return (sigmoid-y).mean(axis=0)

def update(a, g, lr):
    return a - (g * lr)

## Arquetipado del modelo completo

Una vez implementadas las funciones anteriores y realizadas algunas pruebas sencillas de corte numérico, se procederá a implementar el modelo completo de regresión logística en forma de clase, con los métodos `fit` y `predict`, siguiendo así el arquetipo habitual que se viene utilizando a lo largo del curso.

In [85]:
n_iter = 1000
learning_rate = 0.01
b = 0
W = np.zeros(X.shape[1])

lr = 0.01


In [86]:
#Fit
for i in range(n_iter):
    yhat = predict(X, W, b)
    sig = sigmoid(yhat)
    grad_w = dldw(X, y, sig)
    grad_b = dldb(y, sig)
    W = update(W, grad_w, lr)
    b = update(b, grad_b, lr)

In [87]:
# Esqueleto de la clase a implementar


class RegresionLogisticaScratch:
    def __init__(self, learning_rate: float = 0.01, n_iter: int = 1000):

        self.learning_rate = learning_rate
        self.n_iter = n_iter
        self.b = 0
        self.W = None #Se inicializa en el metodo fit

        
    def fit(self, X: np.ndarray, y: np.ndarray):
       
        self.w = np.zeros(X.shape[1])

        for _ in range(self.n_iter):
            yhat = self.__predict(X, self.w, self.b)
            
            sig = self.__sigmoid(yhat)

            loss = self.__loss(y, sig)


            grad_w = self.__dldw(X, y, sig)
            grad_b = self.__dldb(y, sig)

            self.w = self.__update(self.w, grad_w, self.lr)
            self.b = self.__update(self.b, grad_b, self.lr)
    
    
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        yhat = self.__predict(X, self.w, self.b)
        probabilidades = self.__sigmoid(yhat)
        return (probabilidades > 0.5).astype(int)    


    
    @staticmethod
    def __predict(x, w, b):
        return np.dot(x, w) + b

    @staticmethod
    def __sigmoid(yhat):
        return 1/(1+np.exp(-yhat))

    @staticmethod
    def __loss(y, sigmoid):
        return -(y*np.log(sigmoid) + (1-y)*np.log(1-sigmoid)).mean()

    @staticmethod
    def __dldw(x, y, sigmoid):
        return (np.reshape(sigmoid-y, (x.shape[0], 1))*x).mean(axis=0)

    @staticmethod
    def __dldb(y, sigmoid):
        return (sigmoid-y).mean(axis=0)

    @staticmethod
    def __update(a, g, lr):
        return a - (g * lr)

In [88]:
modelo = RegresionLogisticaScratch(n_iter=600, learning_rate=0.1)
modelo.fit(X, y)
ypred = modelo.predict(X)

display(ypred)

array([0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0])